In [ ]:
!git clone https://github.com/kovacsdotgergo/szakdolgozat.git
%cd szakdolgozat
!pip install -r requirements.txt

In [1]:
import utils
esc_path = utils.setup_env()

Running in local


In [2]:
from src.models import ASTModel
import torch
import esc_dataset

have_cuda = torch.cuda.is_available()

input_tdim = 512
audio_model = ASTModel(label_dim=50, input_tdim=input_tdim, imagenet_pretrain=True, audioset_pretrain=True)
audio_model = torch.nn.DataParallel(audio_model, device_ids=[0])
audio_model = audio_model.to(torch.device("cuda:0" if have_cuda else 'cpu'))
audio_model.eval()

AST_NUM_MEL = 128
AST_N_FFT = 1024
AST_HOP_LEN = 256
BATCHSIZE = 16
#instantiating the dataset
dataset = esc_dataset.ESCdataset(esc_path, n_fft=AST_N_FFT, hop_length=AST_HOP_LEN,
                     n_mels=AST_NUM_MEL, augment=False,  log_mel=True,
                     use_kaldi=True, target_len=input_tdim, resample_rate=22500)
spect, label = dataset[5]
spect = spect.unsqueeze(dim=0)
print('elotte')
with torch.no_grad():
    with torch.cuda.amp.autocast():
        out = audio_model.forward(spect)
print('kész')
pred = torch.argmax(out, dim=1)
print(f'pred: {pred == label}')

---------------AST Model Summary---------------
ImageNet pretraining: True, AudioSet pretraining: True
frequncey stride=10, time stride=10
number of patches=600
elotte


: 

: 